# Data Question 3

## Exploring earthquake data

### Nate Silver discusses the difficulty of predicting earthquakes in The Signal and the Noise, but are there factors that make an earthquake more likely?
Are there factors that make an earthquake more deadly?
Where would you live if you wanted to eliminate the risk of experiencing an earthquake?
Where would you avoid living?

## Data cleaning tasks include:
1. Replace empty strings with NaN
2. Remove the footnotes from the 'Other Source Deaths' column
3. Convert Magnitude to a numeric
4. Create a new column ('deaths') that evaluates the four total-death columns ('PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', and 'Other Source Deaths') and populates the new column with the highest value.
5. Explore the data in terms of when and where earthquakes occurred and how severe they were (magnitude, deaths, secondary effects).

In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [32]:
url = 'http://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900'

In [33]:
# create a Request object for the wikipedia page for the Turing Award
response = requests.get(url)

# read the resulting HTML into a variable called result_text
result_text = response.text

In [34]:
# create the soup by constructing a BS object from the html page and the appropriate parser
soup = BS(result_text, 'html.parser')
#soup

In [35]:
# Find table from web page
# There is only one table on the wikipedia page
table = soup.find_all('table')[5]
#table

In [44]:
# Create dataframe
df = pd.read_html(str(table), header=0, index_col = None, parse_dates = True, na_values = 0)

In [50]:
eq_df = df[0]

In [52]:
eq_df.tail()

,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths
1335,2011-03-11 14:46,Japan (see 2011 Tōhoku earthquake),38.322,142.369,24.4,9.0 Mw,T,NaN,NaN,NaN,NaN,15894[20]
1336,2011-03-24 20:25,Burma (see 2011 Burma earthquake),NaN,NaN,NaN,6.8 Mw,NaN,NaN,NaN,NaN,NaN,150+
1337,2011-04-07 14:32,Japan (see April 2011 Miyagi earthquake),38.2,140.0,66.0,7.1 Mw,NaN,NaN,NaN,NaN,NaN,NaN
1338,2011-09-18 12:40,India (see 2011 Sikkim earthquake),27.723,88.064,19.7,6.9 Mw,L,NaN,NaN,NaN,NaN,111+
1339,2011-09-23 10:41,Turkey (see 2011 Van earthquake),38.6,43.5,7.2,7.1 Mw,NaN,NaN,NaN,NaN,NaN,"601 (as of October 30, 2011)"


In [55]:
# Changing columns to make them easier to call
eq_df.columns = ['origin', 'country', 'lat', 'long', 'depth_km', 'magnitude', 'secondary_effects', 
                 'pde_shaking', 'pde_total', 'utsu_total', 'em_dat_total', 'other_source']

eq_df.head()

,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking,pde_total,utsu_total,em_dat_total,other_source
0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN
1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,NaN
2,1900-10-29 09:11,Venezuela,11.000,-66.000,NaN,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN
3,1901-02-15 00:00,China,26.000,100.100,NaN,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN
4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,NaN


In [56]:
eq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 12 columns):
origin               1340 non-null object
country              1340 non-null object
lat                  1326 non-null object
long                 1326 non-null object
depth_km             1160 non-null object
magnitude            1339 non-null object
secondary_effects    373 non-null object
pde_shaking          605 non-null float64
pde_total            721 non-null float64
utsu_total           1027 non-null float64
em_dat_total         543 non-null object
other_source         37 non-null object
dtypes: float64(3), object(9)
memory usage: 125.7+ KB


In [59]:
clean_df = eq_df
clean_df['scale'] = clean_df['magnitude'].str[3:]
clean_df['magnitude'] = clean_df['magnitude'].str[:3]
clean_df.head()

,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking,pde_total,utsu_total,em_dat_total,other_source,scale
0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,
1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9,NaN,NaN,NaN,140.0,NaN,NaN,
2,1900-10-29 09:11,Venezuela,11.000,-66.000,NaN,7.7,NaN,NaN,NaN,NaN,NaN,NaN,
3,1901-02-15 00:00,China,26.000,100.100,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,
4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4,NaN,NaN,NaN,4.0,NaN,NaN,


In [60]:
clean_df['magnitude'] = pd.to_numeric(clean_df['magnitude'])

ValueError: Unable to parse string "5 M" at position 439